# SETUP

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import modules
import numpy as np
import pandas as pd
# import pickle
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [ ]:
#test
path = 'C:/Users/nkalimip/Downloads/Estimate_prediction_process/uncertanity_example/information/ABBY_LAI_trainingdata.csv'
df = pd.read_csv(path)
df = df[["B3",  "B4",  "GVI", "NDVI",  "NLI",  "estimateLAI"]]
print(df)

# #normallize
# from sklearn import preprocessing
# import pandas as pd

# # d = preprocessing.normalize(df)
# scaler = preprocessing.MinMaxScaler()
# d = scaler.fit_transform(df)

# df = pd.DataFrame(d, columns=df.columns)
# print(df)


# pint(n_samples)
# column_number = 1
X,y = df.loc[:, df.columns != 'estimateLAI'],df.loc[:, df.columns == 'estimateLAI']
# X,y = df["B4"], df["estimateLAI"]

# X = np.array([[n] for n in X])
# y = np.array([[n] for n in y])

x_train, x_tst, y_train, y_tst = train_test_split(np.array(X), np.array(y), test_size=0.33)
n_samples = len(x_train)

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

In [ ]:
# neonsl2p:
    
# document where nane[:16] comes from it is an iterator

# Aleatoric Uncertainity

## Defining Log-Likelihood Loss Function

In [21]:
negloglik = lambda y_true, y_pred: -y_pred.log_prob(y_true)

## Defining the Model

In [22]:
for i in biomes:
    normalizer = layers.Normalization(input_shape=[5,], axis=None)
    normalizer.adapt(np.array(calbiomeDictLAI[i]['X_train']))
    
    valbiomeDictLAI[i]['nn'] = tf.keras.Sequential([
        normalizer,
        layers.Dense(80, activation='relu'),
        layers.Dense(25, activation='relu'),
        layers.Dense(tfpl.IndependentNormal.params_size(1)),
        tfpl.IndependentNormal(event_shape=1)])

In [23]:
for i in biomes:
    normalizer = layers.Normalization(input_shape=[5,], axis=None)
    normalizer.adapt(np.array(calbiomeDictLAI[i]['X_train']))
    
    valbiomeDictLAI[i]['nn'] = tf.keras.Sequential([
        normalizer,
        layers.Dense(tfpl.IndependentNormal.params_size(80)),
        tfpl.IndependentNormal(event_shape=80),
        layers.Dense(tfpl.IndependentNormal.params_size(25)),
        tfpl.IndependentNormal(event_shape=25),
        layers.Dense(tfpl.IndependentNormal.params_size(1)),
        tfpl.IndependentNormal(event_shape=1)])
    
     valbiomeDictLAI[i]['nn'].compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
                  loss=negloglik,
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Training the Model

In [ ]:
for i in [1]:
    callbacks = myCallback()
    valbiomeDictLAI[i]['nn_history'] = valbiomeDictLAI[i]['nn'].fit(calbiomeDictLAI[i]['X_train'].to_numpy(), calbiomeDictLAI[i]['y_train'].to_numpy(), epochs=50, validation_split=0.2, verbose=1)

In [ ]:
valbiomeDictLAI[i]['DF']['y_nn'] = valbiomeDictLAI[i]['nn'].predict(valbiomeDictLAI[i]['X_test'])

In [ ]:
y_mean = valbiomeDictLAI[i]['nn'](valbiomeDictLAI[i]['X_test'].to_numpy()).mean()
y_sd = valbiomeDictLAI[i]['nn'](valbiomeDictLAI[i]['X_test'].to_numpy()).stddev()

y_hat_lower = y_mean - 2 * y_sd
y_hat_upper = y_mean + 2 * y_sd

In [ ]:
print(y_sd)
print(y_mean)

In [ ]:
axes = []
for i in [1]:
    axes.append(plt.scatter(valbiomeDictLAI[i]['y_test'], y_sd, s=8, alpha=0.5, edgecolors='none', label=str(i)))
plt.plot([0, 70], [0, 70], 'k--')
plt.xlabel("Reference LAI")
plt.ylabel("Predicted LAI")
plt.title('Neural Network')
plt.xlim([0, 70])
# plt.ylim([-50, 120])
plt.legend(handles=axes, loc='center left', bbox_to_anchor=(1, 0.5), title='Biome')
plt.show()

In [ ]:
sklearn.metrics.mean_squared_error(valbiomeDictLAI[1]['y_test'], valbiomeDictLAI[1]['DF']['y_nn'], squared=False)

In [ ]:
valbiomeDictLAI[2]['nn'].summary()

# Epistemic Uncertainity

## Prior Distribution

In [ ]:
def get_prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
        tfpl.DistributionLambda(lambda t: tfd.MultivariateNormalDiag(
        loc=tf.zeros(n), scale_diag=tf.ones(n)))
    ])
    return prior_model

## Posterior Distribution

In [ ]:
def get_posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
        tfpl.VariableLayer(tfpl.MultivariateNormalTriL.params_size(n), dtype=dtype),
        tfpl.MultivariateNormalTriL(n)
    ])
    return posterior_model

## Defining the model

In [ ]:
# Define the model, negative-log likelihood as the loss function
# and compile the model with the Adam optimizer
normalizer = layers.Normalization(input_shape=[5,], axis=None)
normalizer.adapt(np.array(calbiomeDictLAI[7]['X_train']))

name = 'pnn_8_1'
biome = 7

valbiomeDictLAI[biome][name] = dict()

valbiomeDictLAI[biome][name]['model'] = tf.keras.Sequential([
    normalizer,
    tfpl.DenseVariational(units=8,
                          make_prior_fn=get_prior,
                          make_posterior_fn=get_posterior,
                          kl_weight=1/calbiomeDictLAI[1]['X_train'].shape[0],
                          kl_use_exact=False,
                          activation='relu'),
    # tfpl.DenseVariational(units=25,
    #                       make_prior_fn=get_prior,
    #                       make_posterior_fn=get_posterior,
    #                       kl_weight=1/calbiomeDictLAI[1]['X_train'].shape[0],
    #                       kl_use_exact=False,
    #                       activation='relu'),
    tfpl.DenseVariational(units=1,
                          make_prior_fn=get_prior,
                          make_posterior_fn=get_posterior,
                          kl_weight=1/calbiomeDictLAI[1]['X_train'].shape[0],
                          kl_use_exact=False)
])

def nll(y_true, y_pred):
                   dist = tfp.distributions.Normal(loc=y_pred, scale=1.0)
                   return tf.reduce_sum(-dist.log_prob(y_true))

valbiomeDictLAI[biome][name]['model'].compile(loss=nll, optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=[tf.keras.metrics.RootMeanSquaredError()])
valbiomeDictLAI[biome][name]['model'].summary()

## Train the Model

In [ ]:
# Train the model for 50 epochs
valbiomeDictLAI[biome][name]['history'] = valbiomeDictLAI[biome][name]['model'].fit(calbiomeDictLAI[biome]['X_train'].to_numpy(), calbiomeDictLAI[biome]['y_train'].to_numpy(), epochs=50, verbose=1)
plt.plot(valbiomeDictLAI[biome][name]['history'].history['loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss');

In [ ]:
for i in range(5):
    y_model = valbiomeDictLAI[biome][name]['model'](valbiomeDictLAI[biome]['X_test'])
    plt.scatter(valbiomeDictLAI[biome]['y_test'], y_model, alpha=0.5, s=12, lw=0)
plt.plot([0, 70], [0, 70], 'k--')
plt.xlabel("Reference")
plt.ylabel("Predicted")
plt.title(name)